In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('features_new.csv')

In [150]:
df_final_2 = df[['season','matchday','result','home_rank','away_rank','home_form','away_form','home_GF_pg',
              'home_GA_pg','away_GF_pg','away_GA_pg','home_rank_HT','away_rank_AT','last_conf']]
df_final = df_final_2[(df_final_2.matchday > 20) & (df_final_2.season > 1930)]

In [151]:
X = df_final[['season','matchday','home_rank','away_rank','home_form','away_form','home_GF_pg',
              'home_GA_pg','away_GF_pg','away_GA_pg','home_rank_HT','away_rank_AT','last_conf']]
y = df_final['result']

In [152]:
X

,season,matchday,home_rank,away_rank,home_form,away_form,home_GF_pg,home_GA_pg,away_GF_pg,away_GA_pg,home_rank_HT,away_rank_AT,last_conf
660,1934,21,6,1,0,5,2.450000,2.150000,2.750000,1.400000,8,1,-1
661,1934,21,8,9,-1,0,1.650000,1.900000,2.100000,2.600000,7,4,2
662,1934,21,3,7,3,-1,2.700000,2.000000,1.750000,2.200000,4,10,1
663,1934,21,2,4,4,1,1.800000,0.850000,2.200000,1.700000,1,7,0
664,1934,21,12,10,-5,-1,0.800000,2.450000,1.800000,2.050000,12,9,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48173,2020,42,9,20,-1,-1,0.926829,0.951220,0.926829,1.170732,15,14,1
48174,2020,42,8,2,-3,4,1.048780,1.170732,1.268293,0.634146,11,1,1
48175,2020,42,6,18,2,1,1.268293,0.951220,0.902439,1.292683,8,22,3
48176,2020,42,14,4,1,2,0.902439,0.926829,1.121951,0.780488,9,11,2


In [153]:
from keras.layers import Dense
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [154]:
from sklearn.metrics import accuracy_score

In [155]:
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

X_train,X_test,y_train,y_test = train_test_split(X_scaled,y, test_size = 0.3,
                                                random_state = 20, stratify = y)

In [156]:
targets_a = pd.get_dummies(y_train)
targets_train = targets_a.values
targets_b = pd.get_dummies(y_test)
targets_test = targets_b.values

In [157]:
n_cols = X_train.shape[1]

In [158]:
from keras.callbacks import EarlyStopping

early_stopping_monitor = EarlyStopping(patience = 4)

In [159]:
model = Sequential()
model.add(Dense(100,activation = 'relu', input_shape = (n_cols , )))
#model.add(Dense(100, activation = 'relu'))
#model.add(Dense(10,activation = 'relu'))
#model.add(Dense(10,activation = 'sigmoid'))
#model.add(Dense(10, activation = 'relu'))
#model.add(Dense(100, activation = 'relu'))
#model.add(Dense(100,activation = 'relu'))
#model.add(Dense(100,activation = 'relu'))
#model.add(Dense(100, activation = 'relu'))
#callbacks = [early_stopping_monitor]
model.add(Dense(3,activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(X_train, targets_train, validation_split = 0.3, epochs = 30, callbacks = [early_stopping_monitor])

Epoch 1/30
321/321 [==============================] - 2s 4ms/step - loss: 0.9926 - accuracy: 0.5271 - val_loss: 0.9948 - val_accuracy: 0.5231
Epoch 2/30
321/321 [==============================] - 1s 3ms/step - loss: 0.9794 - accuracy: 0.5323 - val_loss: 0.9896 - val_accuracy: 0.5277
Epoch 3/30
321/321 [==============================] - 1s 3ms/step - loss: 0.9749 - accuracy: 0.5376 - val_loss: 0.9911 - val_accuracy: 0.5286
Epoch 4/30
321/321 [==============================] - 1s 4ms/step - loss: 0.9722 - accuracy: 0.5393 - val_loss: 0.9892 - val_accuracy: 0.5290
Epoch 5/30
321/321 [==============================] - 1s 4ms/step - loss: 0.9705 - accuracy: 0.5401 - val_loss: 0.9892 - val_accuracy: 0.5315
Epoch 6/30
321/321 [==============================] - 1s 4ms/step - loss: 0.9693 - accuracy: 0.5374 - val_loss: 0.9929 - val_accuracy: 0.5240
Epoch 7/30
321/321 [==============================] - 1s 3ms/step - loss: 0.9686 - accuracy: 0.5374 - val_loss: 0.9919 - val_accuracy: 0.5299
Epoch 

In [160]:
pred = model.predict(X_test)
val = pd.DataFrame(pred)

In [161]:
def evaluate_probs(row):
    if row.iloc[0] == max(row.iloc[:]):
        return '1'
    if row.iloc[1] == max(row.iloc[:]):
        return '2'
    else:
        return 'X'

In [162]:
val['pred'] = val.apply(evaluate_probs,axis = 1)
pred_final = val['pred']

In [163]:
pred_final.value_counts()

1    5557
2     684
X      34
Name: pred, dtype: int64

In [164]:
accuracy_score(y_test, pred_final)

0.5311553784860558